<a href="https://colab.research.google.com/github/nithinsrinivas1993/SUMO-based-project/blob/master/SUMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from subprocess import check_output
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from sklearn.model_selection import  train_test_split
import time #helper libraries
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from numpy import newaxis
from geopy.geocoders import Nominatim

In [0]:
pip install keras 

In [0]:
pip install geopy 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
dataset = pd.read_csv("/content/drive/My Drive/Datasheet_clean.csv")

dataset = dataset.dropna()
dataset = dataset.drop_duplicates()
print(dataset)
#data.to_csv('/content/drive/My Drive/Datasheet_clean.csv')

      Unnamed: 0  Unnamed: 0.1  Timestamp  ...  VehiclePos.1 Longitude   latitude
0              0             0          0  ...      4.899500 -6.261524  53.352844
1              1             1          1  ...      3.904634 -6.261489  53.352816
2              2             2          2  ...      0.349166 -6.261486  53.352814
3              3             3          3  ...      0.041667 -6.261485  53.352814
4              4             4          4  ...      0.002291 -6.261485  53.352814
...          ...           ...        ...  ...           ...       ...        ...
4731        4731          5384       3734  ...     14.039617 -6.251481  53.348799
4732        4732          5385       3735  ...     15.545515 -6.251473  53.348657
4733        4733          5386       3736  ...     16.167585 -6.251479  53.348509
4734        4734          5387       3737  ...     11.667585 -6.251402  53.348418
4735        4735          5388       3738  ...     14.041988 -6.251285  53.348315

[4736 rows x 9 

In [0]:
dataset

,Unnamed: 0,Unnamed: 0.1,Timestamp,Vehicle Angle,VehicleID,VehiclePos,VehiclePos.1,Longitude,latitude
0,0,0,0,140.695093,veh0,5.100000,4.899500,-6.261524,53.352844
1,1,1,1,140.695093,veh0,9.004634,3.904634,-6.261489,53.352816
2,2,2,2,140.695093,veh0,9.353800,0.349166,-6.261486,53.352814
3,3,3,3,140.695093,veh0,9.395467,0.041667,-6.261485,53.352814
4,4,4,4,140.695093,veh0,9.397758,0.002291,-6.261485,53.352814
...,...,...,...,...,...,...,...,...,...
4731,4731,5384,3734,166.581883,veh72,18.571819,14.039617,-6.251481,53.348799
4732,4732,5385,3735,177.287754,veh72,34.117334,15.545515,-6.251473,53.348657
4733,4733,5386,3736,179.161726,veh72,50.284919,16.167585,-6.251479,53.348509
4734,4734,5387,3737,141.330902,veh72,3.342503,11.667585,-6.251402,53.348418


In [0]:
dataset.head()

,Unnamed: 0,Unnamed: 0.1,Timestamp,Vehicle Angle,VehicleID,VehiclePos,VehiclePos.1,Longitude,latitude
0,0,0,0,140.695093,veh0,5.100000,4.899500,-6.261524,53.352844
1,1,1,1,140.695093,veh0,9.004634,3.904634,-6.261489,53.352816
2,2,2,2,140.695093,veh0,9.353800,0.349166,-6.261486,53.352814
3,3,3,3,140.695093,veh0,9.395467,0.041667,-6.261485,53.352814
4,4,4,4,140.695093,veh0,9.397758,0.002291,-6.261485,53.352814


In [0]:
print(len(dataset[["latitude","Longitude" , "Vehicle Angle"]]))

4736


In [0]:
final_dataset = dataset[["latitude","Longitude","Vehicle Angle"]]

In [0]:
final_dataset

,latitude,Longitude,Vehicle Angle
0,53.352844,-6.261524,140.695093
1,53.352816,-6.261489,140.695093
2,53.352814,-6.261486,140.695093
3,53.352814,-6.261485,140.695093
4,53.352814,-6.261485,140.695093
...,...,...,...
4731,53.348799,-6.251481,166.581883
4732,53.348657,-6.251473,177.287754
4733,53.348509,-6.251479,179.161726
4734,53.348418,-6.251402,141.330902


In [0]:
train_size = int(len(final_dataset) * 0.80)
test_size = len(final_dataset) - train_size
train, test = final_dataset[0:train_size], final_dataset[train_size:len(final_dataset)]
print(len(train), len(test))

3788 948


In [0]:
#convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back)]
        dataX.append(np.array(a))
        dataY.append(np.array(dataset[i + look_back:i+look_back+1]))
        
    return np.array(dataX),np.array(dataY)

In [0]:
look_back = 50
trainX, trainY = create_dataset(train, look_back)

In [0]:
print(trainX.shape)
print(trainY.shape)

(3737, 50, 3)
(3737, 1, 3)


In [0]:
#Step 2 Build Model
model = Sequential()
model.reset_states()

model.add(LSTM(
    input_dim=3,
    output_dim=100,
    return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(
    100,
    return_sequences=True))
model.add(Dropout(0.2))

model.add(Dense(
    output_dim=3))
model.add(Activation('linear'))

start = time.time()
model.compile(loss='mse', optimizer='adam')
print ('compilation time : ', time.time() - start)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(return_sequences=True, input_shape=(None, 3), units=100)`
  


compilation time :  0.012531757354736328


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=3)`


In [0]:
model.fit(
    trainX,
    trainY,
    batch_size=len(trainX),
    nb_epoch=100,
    shuffle = False,
    validation_split=0.05)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys


Train on 3550 samples, validate on 187 samples
Epoch 1/100
3550/3550 [==============================] - 6s 2ms/step - loss: 12966.7480 - val_loss: 30793.1035
Epoch 2/100
3550/3550 [==============================] - 6s 2ms/step - loss: 12923.8955 - val_loss: 30719.7871
Epoch 3/100
3550/3550 [==============================] - 6s 2ms/step - loss: 12877.2471 - val_loss: 30639.8867
Epoch 4/100
3550/3550 [==============================] - 6s 2ms/step - loss: 12826.4639 - val_loss: 30552.5703
Epoch 5/100
3550/3550 [==============================] - 6s 2ms/step - loss: 12771.2500 - val_loss: 30458.7559
Epoch 6/100
3550/3550 [==============================] - 6s 2ms/step - loss: 12712.0742 - val_loss: 30362.0762
Epoch 7/100
3550/3550 [==============================] - 6s 2ms/step - loss: 12651.2080 - val_loss: 30266.6953
Epoch 8/100
3550/3550 [==============================] - 6s 2ms/step - loss: 12590.4580 - val_loss: 30175.3398
Epoch 9/100
3550/3550 [==============================] - 6s 2ms/s

In [0]:
test.shape
testX, testY = create_dataset(test, look_back)

In [0]:
test1 = np.array(testX[0:1])
print(test1[0:1][0:1])


[[[ 53.352207  -6.262319 227.93251 ]
  [ 53.352842  -6.255939 335.387673]
  [ 53.352165  -6.262402 227.93251 ]
  [ 53.352938  -6.256005 335.387673]
  [ 53.352123  -6.262487 228.024202]
  [ 53.352992  -6.256074 312.617726]
  [ 53.352085  -6.262565 228.336497]
  [ 53.352981  -6.256179 250.850975]
  [ 53.352047  -6.262643 228.783772]
  [ 53.352952  -6.256314 248.781875]
  [ 53.349756  -6.260076 163.45156 ]
  [ 53.352002  -6.262729 226.271674]
  [ 53.352926  -6.256443 248.781875]
  [ 53.349742  -6.26007  163.45156 ]
  [ 53.351956  -6.262808 223.640211]
  [ 53.352897  -6.25658  248.781875]
  [ 53.349711  -6.260057 163.45156 ]
  [ 53.351919  -6.262883 229.15418 ]
  [ 53.352869  -6.256717 248.781875]
  [ 53.349668  -6.260039 163.45156 ]
  [ 53.351915  -6.262981 269.982114]
  [ 53.352842  -6.256847 248.444073]
  [ 53.349605  -6.260012 163.45156 ]
  [ 53.351936  -6.263072 292.326781]
  [ 53.352813  -6.256983 248.344478]
  [ 53.349524  -6.259977 163.45156 ]
  [ 53.351968  -6.263165 298.050431]
 

In [0]:
model.predict(test1)

array([[[11.533971 , -4.5467663, 15.409312 ],
        [15.061231 , -6.0033712, 20.101568 ],
        [15.580061 , -6.216056 , 20.790201 ],
        [15.651137 , -6.2446766, 20.885736 ],
        [15.661651 , -6.249237 , 20.899223 ],
        [15.662505 , -6.2493134, 20.900963 ],
        [15.663223 , -6.249877 , 20.90135  ],
        [15.663129 , -6.249762 , 20.9014   ],
        [15.663264 , -6.249891 , 20.901423 ],
        [15.66315  , -6.249771 , 20.901426 ],
        [15.663758 , -6.250391 , 20.901476 ],
        [15.663562 , -6.250188 , 20.90146  ],
        [15.663439 , -6.250063 , 20.901453 ],
        [15.663955 , -6.2505903, 20.901497 ],
        [15.6637945, -6.250428 , 20.901463 ],
        [15.663667 , -6.2503014, 20.901453 ],
        [15.664087 , -6.250725 , 20.90151  ],
        [15.663923 , -6.2505636, 20.90146  ],
        [15.663821 , -6.2504616, 20.90145  ],
        [15.664166 , -6.250806 , 20.901516 ],
        [15.663767 , -6.250415 , 20.901438 ],
        [15.66391  , -6.250559 , 2